In [ ]:
library(rmd2jupyter)
rmd2jupyter("US_Flight_Analysis_Notebook.Rmd")


## Introduction

This project is intended to demonstrate the skills acquired from the Google Data Analytics Certificate Course hosted on [Coursera](https://www.coursera.org/professional-certificates/google-data-analytics). The data set was retrieved from [Kaggle](%22https://www.kaggle.com/datasets/undersc0re/flight-delay-and-causes/metadata?select=Flight_delay.csv%22). Originally, the data set comes form the [U.S. Department of Transportation's (DOT) Bureau of Transportation Statistics (BTS)](https://www.bts.gov/).

A description for the original column labels can be looked up by clicking the following [link](https://www.kaggle.com/datasets/undersc0re/flight-delay-and-causes?select=Flight_delay.csv).

The attempt to analyze the data set in a Spreadsheet (Excel) failed due to its high volume. I personally decided to use R over SQL because R is more functional and also allows me to visualize the data.

\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_

## General Analysis

### Data Preparation

#### **1** Loading the required packages for the analysis

If the packages are not installed yet, use the install.packages() function first!

Note that the library plyr has to be loaded prior to dplyr to prevent any issues


In [ ]:
library(tidyverse)
library(janitor)
detach("package:plyr") # detaching both libraries ...
detach("package:dplyr")
library(plyr) # ... and loading them again to make sure 
library(dplyr) # they are loaded in the right order
library(readr)
library(lubridate)
library(ggcorrplot)
library(RColorBrewer)
library(sqldf)
library(scales)
library(ggpubr)
library(ggcorrplot)


------------------------------------------------------------------------

#### **2** Opening the data set


In [ ]:
# download data from 
# < http://127.0.0.1:56595/%22https://www.kaggle.com/datasets/undersc0re/flight-delay-
# and-causes/metadata?select=Flight_delay.csv%22 >
# and assign the path as a string to the local_path variable
local_path <- ".../Flight_delay.csv"
flights_df <- read_csv(local_path)


In [ ]:
head(flights_df)



------------------------------------------------------------------------

#### **3** For the sake of visual appeal, I renamed the column names and converted them all to lowercase


In [ ]:
names(flights_df) <- tolower(names(flights_df %>% 
                                     dplyr::rename(weekday = DayOfWeek,
                                            dep_time = DepTime,
                                           arr_time = ArrTime,
                                           scheduled_arr_time = CRSArrTime, 
                                           uniq_carrier_code = UniqueCarrier,
                                           flight_num = FlightNum,
                                           tail_num = TailNum,
                                           actual_flight_time_min = ActualElapsedTime,
                                           estimate_flight_time_min = CRSElapsedTime,
                                           air_time_min = AirTime,
                                           arr_delay = ArrDelay,
                                           dep_delay = DepDelay,
                                           dep_airport_code = Origin,
                                           dep_airport = Org_Airport,
                                           dest_airport_code = Dest,
                                           dest_airport = Dest_Airport,
                                           distance_miles = Distance, 
                                           landing_to_gate_min = TaxiIn,
                                           gate_to_takeoff_min =TaxiOut,
                                           cancellation_cause_code = CancellationCode,
                                           carrier_delay = CarrierDelay,
                                           weather_delay = WeatherDelay,
                                           nas_delay = NASDelay,
                                           security_delay = SecurityDelay,
                                           late_aircraft_delay = LateAircraftDelay)))


In [ ]:
colnames(flights_df)



------------------------------------------------------------------------

#### **4** Next, we remove columns that don't give us any information (due to a lack of data)


In [ ]:
vector <- c()
for (i in names(flights_df)) {
  if (is_double(flights_df[[i]][2]) == TRUE) {
    if (sum(flights_df[i]) == 0 ) {
      vector <- append(vector, i)
    }
  }
}


In [ ]:
cat("the vector contains columns:", vector, sep="\n-")



When we investigate the columns "cancelled" and "diverted", they only contain 0!

Let's get rid of the two unnecessary columns (2 methods)


In [ ]:
#method 1: selecting all except from elements of vector
flights_df <- select(flights_df, -all_of(vector))


In [ ]:
#method 2: dropping useless columns
flights_df <- flights_df[!(names(flights_df) %in% vector)]


------------------------------------------------------------------------

#### **5** Creating a new column that contains values of the total delay for each specific flight


In [ ]:
flights_df <- mutate(flights_df,
                     total_delay = (carrier_delay + weather_delay + nas_delay + 
                                    security_delay + late_aircraft_delay))


------------------------------------------------------------------------

#### **6** Creating a new column that contains the month each individual flight took place


In [ ]:
library(lubridate)

flights_df <- flights_df %>% mutate(month = month(dmy(date)))


In [ ]:
colnames(flights_df)



------------------------------------------------------------------------

#### **7** Determining, in which delay category each flight falls

I classified the delay according to the Federal Aviation Administration (FAA) that considers an actual arrival less than 15 min after the scheduled arrival as not delayed, an arrival between 15 and 45 min after the scheduled arrival as "medium delay" and beyond 45 min as "large delay". Source: [Wikipedia](https://en.wikipedia.org/wiki/Flight_cancellation_and_delay#:~:text=A%20flight%20delay%20is%20when,all%20for%20a%20certain%20reason)


In [ ]:
flights_df <- flights_df %>% 
  mutate(degree_delay =
           ifelse(total_delay <= 15, "no delay", 
                  ifelse(total_delay >= 45, "large delay", "medium delay")))


Having learned Python as a first programming language, I love to write loops, functions and conditional statements. In this case, it was a tedious mistake to apply Python practices to R:

Technically, this can be done with a for-loop and conditional statements too; however, the computing time is awfully long with bigger data frames (30-40 min) since functions in R usually do not directly modify the data frame, but instead making copies. For every single iteration, R therefore makes a copy of the entire data frame! Fortunately, I found help on [Stack Overflow](https://stackoverflow.com/questions/22337394/dplyr-mutate-with-conditional-values).


In [ ]:
vec <- c()
for (t in flights_df$total_delay) {
  if (t <= 15) {
    vec <- append(vec, "No delay")
  }
  if (t >= 45) {
    vec <- append(vec, "Large delay")
  }
  else {
    vec <- append(vec, "Medium delay")
  }
}

# Creating a new column from the vector containing 
# the categorization of each flight
flights_df["delay_degree"] <- vec


------------------------------------------------------------------------

#### **8** This step is mainly for the sake of practicing Data Manipulation

(this case does not apply to the US since it is an EU law):

creating a new column which states whether the passenger are potentially subject to compensation according to EU261 law. Passengers are eligible to claim up to 600€ as soon as the flight is delayed for 3 hours, and receive a full refund, if delayed for 5 hours or longer.


In [ ]:
flights_df <- flights_df %>% 
  mutate(compensation =
           ifelse(total_delay < 180, "no compensation", 
                  ifelse(total_delay >= 300, "full refund", "up to 600€")))


As with the previous step, this code using the for-loop is highly inefficient. I still left it because it is technically correct viewing it from a logical perspective :)



In [ ]:
vect <- c()
for (c in flights_df$total_delay){
  if (c < 180){
    vect <- append(vect, "no compensation")
  }
  if (c >= 300){
    vect <- append(vect, "full refund")
  }
  else {
    vect <- append(vect, "up to 600€")
  }
}
flights_df["compensation"] <- vect


Let's have a look at the structure of our final data frame:



In [ ]:
glimpse(flights_df)



------------------------------------------------------------------------

## Data Exploration

#### **9** Let's compute, which airline has the most delay time in the given time frame


In [ ]:
flights_df %>% 
    dplyr::group_by(airline) %>% 
    drop_na() %>% 
    summarize(accumulated_delay = sum(total_delay)) %>% 
    arrange(-accumulated_delay)


So far, so good. But simply concluding that Southwest Airline Co. is the least reliable Airline would be *false* since Southwest operates the most flights in the given time period.

To demonstrate this, let's compute, and then display the number of flights of each individual airline.


In [ ]:
as.data.frame(table(flights_df$airline)) %>% arrange(-Freq)



**ggplot2** is an awesome and handy package for data visualization



In [ ]:
ggplot(flights_df) +
  geom_bar(aes(x = airline), fill = "#00CC99", color = "#009933", alpha = 0.7) +
  theme(axis.text.x = element_text(angle = 35, hjust = 1)) + 
  labs(title = "Number of Flights US Airlines",
        x = "Airline", y = "Number of Flights")


In [ ]:
ggplot(flights_df) +
  geom_bar(aes(x = airline), fill = "#00CC99", color = "#009933", alpha = 0.7) +
  theme(axis.text.x = element_text(angle = 35, hjust = 1)) + 
  labs(title = "Number of Flights US Airlines",
        x = "Airline", y = "Number of Flights") +
  facet_wrap(~month)


A better measure would be the average (or mean) delay for each airline.



In [ ]:
flights_df %>% 
  group_by(airline) %>% 
  drop_na() %>% 
  summarize(delay = mean(total_delay)) %>% 
  arrange(-delay)


------------------------------------------------------------------------

#### **10** Next, let's explore, what is the biggest driver for delay?


In [ ]:
flights_df %>%  summarize(total_carrier = sum(carrier_delay),
                          total_weather = sum(weather_delay),
                          total_nas = sum(nas_delay),
                          total_security = sum(security_delay),
                          total_late_aircraft = sum(late_aircraft_delay)) %>% 
  pivot_longer(cols=1:5, names_to = 'Delay_Type', values_to = 'Accumulated_Delay') %>% 
  arrange(-Accumulated_Delay)


Are we still getting the same ranting if we compare the accumulated delay of each delay type to the average delay?



In [ ]:
df1 <- flights_df %>%  summarize(carrier = sum(carrier_delay),
                          weather = sum(weather_delay),
                          nas = sum(nas_delay),
                          security = sum(security_delay),
                          late_aircraft = sum(late_aircraft_delay)) %>% 
  pivot_longer(cols=1:5, names_to = 'Delay_Type', values_to = 'Accumulated_Delay') %>% 
  arrange(-Accumulated_Delay)

df2 <- flights_df %>% summarize(carrier = mean(carrier_delay),
                      weather = mean(weather_delay),
                      nas = mean(nas_delay),
                      security = mean(security_delay),
                      late_aircraft = mean(late_aircraft_delay)) %>% 
  pivot_longer(cols=1:5, names_to = 'Delay_Type', values_to = 'Average_Delay') %>% 
  arrange(-Average_Delay)

#inner join of both data frames by the primary key 'Delay_Type'
merge(df1, df2) %>% arrange(-Average_Delay) 


In [ ]:
merge(df1, df2) %>% 
  arrange(-Average_Delay) %>%  
  pivot_longer(cols = c("Accumulated_Delay", "Average_Delay"), 
               names_to ="Method", values_to = "Value") %>% 
  ggplot() + 
  geom_bar(aes(x = reorder(Delay_Type, -Value), y = Value, fill = Delay_Type), 
           color = "dark grey", alpha = 0.9, stat="identity", position = "dodge") + 
  facet_wrap(~Method, scale = "free") + 
  scale_y_continuous(labels = format_format(big.mark = ",", scientific = FALSE)) +
  labs(x = "Delay Type", y = "Delay (min)", fill = "") +
  theme(legend.position="top", axis.text.x = element_blank(), axis.ticks.x = element_blank()) +
  scale_fill_brewer(palette = 14)


Let's come back to the average flight delay - How big are the differences in the average flight delay if we compare the 12 airlines to each other?



In [ ]:
avg <- flights_df %>% 
  group_by(airline) %>% 
  drop_na() %>% 
  summarize(delay = mean(total_delay)) %>% 
  arrange(-delay)

avg


Let's visualize the code by using another graph!



In [ ]:
startdate <-  min(flights_df$date)
enddate <-  max(flights_df$date)

ggplot(data=avg) +
  geom_bar(aes(x = stats::reorder(airline, -delay), y = delay, fill = airline), 
           stat = "identity", width = 0.6) +
  labs(title = "Average Delay per Airline", subtitle = paste("From", startdate, "to", enddate),
       caption = "by Markus Köfler", x = "Airlines", y = "Average Delay (min)") +
  theme(axis.text.x = element_blank()) + 
  theme(axis.text.x = element_text(angle = 45, hjust = 1)) 


Or displaying the average delay of Airlines for each month - maybe we can get even better insights from the data?!



In [ ]:
ag <- flights_df %>% 
  group_by(airline, month) %>% 
  drop_na() %>% 
  summarize(delay = mean(total_delay)) 


ggplot(data=ag) +
  geom_bar(aes(x = reorder(airline, -delay), y = delay, fill = airline), 
           stat = "identity", width = 0.6) +
  labs(title = "Average Delay per Airline", subtitle = paste("From", startdate, "to", enddate),
       caption = "by Markus Köfler", x = "Airlines", y = "Average Delay (min)") +
  theme(axis.text.x = element_blank()) + 
  theme(axis.text.x = element_text(angle = 70, hjust = 1)) +
  facet_wrap(~month)
  


We can see that Alaska Airlines average delay for June is 0 min. Can Alaska Airlines really boast that none of their flights was delayed in June or are there just no recorded flights?



In [ ]:
nrow(filter(flights_df, airline=="Alaska Airlines Inc." & month==6))



As the output suggests, the returned tibble contains 0 rows, meaning that there is no data on Alaska Airline flights in June. Further research needs to be done with regards to why this is the case.

------------------------------------------------------------------------

\####**12** The relationship between the total delay and the flight distance - can passengers expect a longer delay for longer travels? We can also add the correlation coefficients with p-values to the scatter plot


In [ ]:
ggplot(flights_df) +
  geom_jitter(aes(distance_miles, total_delay), alpha = 0.1, shape = "o", color = "navy") +
  geom_smooth(aes(distance_miles, total_delay), color = "red", method = "lm") + 
  facet_wrap(~airline, scale = "free", shrink = FALSE) + #adjusted x- and y-axis 
  stat_cor(aes(distance_miles, total_delay), 
           color = "red", geom = "label", fill = "transparent") +
  labs(title = "Corellation between Flight Distance and Total Delay", 
       subtitle = "Individual Airlines",
       caption = "by Markus Köfler", x = "Distance (miles)", y = "Delay (min)") +
  theme(axis.text.x = element_text(angle = 20, hjust = 1)) 


In [ ]:
#relationship between delay and flight duration/ distance (do longer trips mean a longer expected delay?)

ggplot(flights_df) +
  geom_jitter(aes(distance_miles, total_delay), shape = "+", alpha = 0.9) +
  geom_smooth(aes(distance_miles, total_delay), color = "red") + 
  labs(title = "Overall Correlation between Flight Distance and Total Delay", 
       subtitle = paste("Correlation:",
                        toString(cor(flights_df$distance_miles, flights_df$total_delay)), 
                        sep = " "))


------------------------------------------------------------------------

#### **12** Now lets find out what are the most popular arrival and departure airports


In [ ]:
dep_airport_df <- dplyr::rename(as.data.frame(table(flights_df$dep_airport)) %>% 
  arrange(-Freq), dep_airport = Var1, departures = Freq)

dest_airport_df <- dplyr::rename(as.data.frame(table(flights_df$dest_airport)) %>% 
  arrange(-Freq), dest_airport = Var1, arrivals = Freq)

dep_dest_airports <- cbind(dep_airport_df, dest_airport_df)

head(dep_dest_airports, n = 10)


The created data frame tells us, what are the airports with the most (domestic) traffic. A tendency, that airports with the most departures also rank high when it comes to arrivals, is given. Let's investigate the correlation between the departure rank and the arrival rank:



In [ ]:
len_of_df <- length(dep_dest_airports$dep_airport)

# assigning integers from 1 to 260
rank <- c(1:len_of_df) 

# adding ranking to each individual data frame
dep_rank_df <- mutate(dplyr::rename(dep_airport_df, airport = dep_airport), rank_dep = rank)
dest_rank_df <- mutate(dplyr::rename(dest_airport_df, airport = dest_airport), rank_dest = rank)

#library(plyr)
# joining the data frames based on a common key which is the column "airport" 
dep_dest_rank <- arrange(plyr::join(dep_rank_df, 
                                    dest_rank_df, type = "full", 
                                    by = "airport"), 
                         + rank_dep)

top_n(dep_dest_rank, -10)


Now that we have the ranking for departures and arrivals, we can compute the correlation. I used the 3 common [correlation methods](https://www.phdata.io/blog/data-science-stats-review/):

-   Pearson =\> linear relationship between two variables

-   Kendall =\> monotonic relationship (likelihood of two variables to move in one direction, but not necessarily in a constant manner)

-   Spearman =\> monotonic relationship (similar to Kendall method, but not as popular)


In [ ]:
# computing the correlation
# function which iterates through a vector containing
# the 3 correlation methods used in data science
cor_methods <- c("pearson", "kendall", "spearman")

for (cor_method in cor_methods) {
     print(paste(cor_method, sep = ": ", 
                cor(dep_dest_rank$rank_dep, dep_dest_rank$rank_dest, method = cor_method)
                )
           )
}


Here is a much more sophisticated syntax. I did this to make my code more reproducible. Next time I want to compute the statistical correlation with all 3 methods, I simply call the function and pass in the arguments for the parameters var1 and var2.



In [ ]:
cor_calculator <- function (method_vector = c("pearson", "kendall", "spearman")
                            , var1, var2) { 
  result <- c()
  for (cor_method in method_vector) {
    result <- append(result, paste(cor_method, sep = ": ", 
                cor(dep_dest_rank$rank_dep, dep_dest_rank$rank_dest, method = cor_method)))
    }
  return(result)
}


variable_1 <- dep_dest_rank$rank_dep
variable_2 <- dep_dest_rank$rank_dest

cor_calculator(var1 = variable_1, var2 = variable_2)


------------------------------------------------------------------------

#### **13** What are the most frequent routes flown in the US from January to June 2019?

To answer this question, I combined the columns dep_airport and dest_airport to build a column which contains both departure airport as well as destination airport. This allows us to get unique flight routes.


In [ ]:
flights_df["dep_dest_airports"] <- paste("FROM:", flights_df$dep_airport, 
                                         "TO:", flights_df$dest_airport, 
                                         sep = " ")

flights_df$dep_dest_airports[1:5]


The next step is counting what unique flight route occurs the most in the newly created column. Finally, we can arrange the data frame in descending order.



In [ ]:
routes_df <- as.data.frame(table(flights_df["dep_dest_airports"])) %>% arrange(-Freq)

# display the top 10 mosth frequent travel routes 
top_n(routes_df, 10)


------------------------------------------------------------------------

#### **14** At the end of the general analysis I always like to add a

correlation matrix. The intention is to highlight possible relationships and trends between variables that have not been discovered yet.


In [ ]:
#filtering for columns that are numeric only

flights_numeric <- select_if(flights_df, is.numeric)

# Computing correlation matrix
cor_matrix <- round(cor(flights_numeric),3)

# Visualizing and reordering correlation matrix
ggcorrplot(cor_matrix, hc.order =FALSE, tl.cex = 8,
           outline.color ="#808080", method = "square", colors = c("#FF007F", "white", "#0000FF")) +
  labs(title= "Correlation Matrix") +
  theme(plot.title = element_text(size = 22, hjust = 1)) 


Based on den matrix, there is nothing outstanding to report.

*Strongly positively* related are:

-   flight distance (distance_miles) with the air time (air_time_min), the estimated flight time (estimate_flight_time_min) and the actual flight time (actual_flight_time_min)

-   departure delay with the arrival delay

-   the total delay (total_delay) with the departure delay (dep_delay) and the arrival delay (arr_delay)

Optionally, we can compute the correlation matrix in numbers with p-values with the following code:


In [ ]:
corrp.mat <- cor_pmat(flights_numeric)
corrp.mat


------------------------------------------------------------------------

## The Business Task

1)  A business consultancy company is sending their consultants to their customers within the US area (domestic flights).

2)  The consultancy company is located in Chicago (IL)

3)  Senior consultant Andrew needs to fly to a client located in Los Angeles. He passes his appointment to the HR team, which takes over responsibility for managing client meetings and travels for employees. HR manager Thomas asks for an analysis, what would be the best option to go from Chicago to Dallas.

We start preparing the data frame first - we create a column with the flight routes. This time, we only use Airport codes which consist of 3 uppercase letters to make the the script more readible:


In [ ]:
flights_df <- mutate(flights_df, 
                     route = paste(flights_df$dep_airport_code, 
                                 flights_df$dest_airport_code, 
                                 sep = "-"))

flights_df$route[1:5]


For finding the routes with the shortest average delay that can be expected (based on the data), I used SQL statements by using the library **sqldf**. It allows us to query the data frame in SQL-syntax style by passing in the SQL statement as a string.

#### SQL query


In [ ]:
sqldf("
       SELECT 
          route,
          airline,
          avg(actual_flight_time_min) AS average_travel_time,
          avg(total_delay) AS average_delay
       
       FROM
          flights_df
      
      WHERE 
        route = 'ORD-LAX' OR route = 'MDW-LAX' 
      
      GROUP BY 
          airline
      
      ORDER BY
          average_delay ASC
      ")
     


According to the results, the best option would be to book a flight from Chicago Midway (MDW) to LA International (LAX) in terms of expected reliability. The differences in average travel time is too insignificant and can be neglected.

Next, a consultant, who has been negotiating with a client in Dallas (TX) needs to directly visit a nother customer in New York. There are three target airports in NY to choose from at the time. There is also the option to either leave from Dallas Fort-Worth or Dallas Love Fields. What is the best constellation of airports to choose from?


In [ ]:
sqldf("
      SELECT 
         airline,
         route,
         avg(actual_flight_time_min) AS average_travel_time,
         avg(total_delay) AS average_delay
     
      FROM
        flights_df
     
     WHERE 
         route = 'DFW-JFK' OR 
         route = 'DFW-LGA' OR 
         route = 'DFW-EWR' OR 
         route = 'DAL-JFK' OR
         route = 'DAL-LGA' OR
         route = 'DAL-EWR' 
    
     GROUP BY 
         route 
    
     ORDER BY 
         average_travel_time ASC
       ") 


The results suggest that DFW has better connection to one of the popular NYC airports (since there are no other flights recorded from Dallas Love Fields). We assume that DFW has better flight schedules to NYC. When it comes to choosing an airport in NYC, we have to make a trade-off whether to accept a slightly higher average travel delay to have an overall shorter expected travel time.

Just to be certain - we check if there are really no flights from DAL to any NYC airport in our data set.


In [ ]:
sum((flights_df$dep_airport_code == "DAL" & flights_df$dest_airport_code == "JFK") |
    (flights_df$dep_airport_code == "DAL" & flights_df$dest_airport_code == "LGA") |
    (flights_df$dep_airport_code == "DAL" & flights_df$dest_airport_code == "EWR"))



Indeed, we cannot find any flights from Dallas Love Fields to a NYC airport.
